In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import string

from collections import Counter

import nltk, re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from spellchecker import SpellChecker

## Data Analysis steps:

<ol>
    <li> Define the question  </li>
    <li> Define the ideal data set </li>
    <li> Obtain data </li>
    <li> Clean the data </li>
    <li> Exploratory Data Analysis </li>
    <li> Statistical Prediction/modeling </li>
    <li> Interpret results </li>
    <li> Challenge results </li>
    <li> Communicate results </li>
    <li> Build a Data Product </li>
</ol>


### Step 1: Define the Question(s)

<ul>
    <li>What was the most common issue a customer had towards airlines? </li>
    <li> Which Airlines had the most staisfying responses from customers? How fast were they to respond? </li> 
</ul>

### Step 2 & 3: Identify and Obtain the Data

<div> Import Dataset </div>

In [2]:
df = pd.read_csv('twcs.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


<hr>
<h3> Dataset contains information about discussion between a customer and company chatbots. </h3>
<p> tweet_id: Unique ID for tweet => Generates every tweet</p>
<p> author_id: Unique username of the author </p>
<p> inbound: True => Represents the customer sending message; False => Represents chatbot responses </p>
<p> created_at: Date at which post/message was created or sent </p>
<p> text: Message/Text sent </p>
<p> response_tweet_id: tweet_id(s) that responded to the current tweet </p>
<p> in_response_to_tweet_id: tweet_id(s) that the current tweet is responding to </p>
<hr>

In [3]:
df.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 This is saddening to hear. Please shoo...,NaN,12.0
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare You gonna magically change your co...,"11,13,14",15.0
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 We understand your concerns and we'd l...,12,16.0


### Step 4: Clean the Data

<hr>

<h4> Ideal Dataset Contains the following: </h4>

<ul>
    <li> Filtered to only Airlines </li>
    <li> Cleaned text data to simpliest words </li>
    <li> Data is wrangled to show original message and response in same row </li>
    <li> Correct data types for all columns </li>
    <li> New Column to identify sentiment of the text </li>
    <li> Fill in null values </li>
    <li> Eliminate duplicate values </li>
</ul>

<hr>

<h4> Clean Text Data </h4>

<ul>
    <li> Isolate text data </li>
    <li> Lower case all words </li>
    <li> Remove stop words (i, me, why, etc.) </li>
    <li> Remove puncuation </li>
    <li> Remove common words </li>
    <li> Remove rare words </li>
    <li> Remove emojis </li>
    <li> Remove URLs </li>
</ul>

<div> Download nltk packages to help with cleaning </div>

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<div> Isolate Text Data </div>

In [5]:
textDF = df[['text']]
textDF.head(5)

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


<div> Change all words to lowercase </div>

In [6]:
textDF['text_lower'] = textDF.loc[:,'text'].str.lower()
textDF.head(5)

C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\3470515980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_lower'] = textDF.loc[:,'text'].str.lower()


,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


<div> Remove Puncuation </div>

In [7]:
punctuations = string.punctuation
print(punctuations)

%timeit
def remove_punctuations(text):
    return text.translate(str.maketrans('','',punctuations))
    

textDF['text_wo_punctuations'] = textDF['text_lower'].apply(lambda text: remove_punctuations(text))
textDF.head(5)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\545002421.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_punctuations'] = textDF['text_lower'].apply(lambda text: remove_punctuations(text))


,text,text_lower,text_wo_punctuations
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did


<div> Remove Emojis </div>

In [8]:
emojiPattern = re.compile("["u"\U0001F600-\U0001F64F""]+", flags=re.UNICODE)
textDF['text_wo_emojis'] = textDF['text_wo_punctuations'].apply(lambda text: emojiPattern.sub(r'', text))
textDF.tail(5)

C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\3645892192.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_emojis'] = textDF['text_wo_punctuations'].apply(lambda text: emojiPattern.sub(r'', text))


,text,text_lower,text_wo_punctuations,text_wo_emojis
2811769,"@823869 Hey, we'd be happy to look into this f...","@823869 hey, we'd be happy to look into this f...",823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...
2811770,@115714 wtf!? I’ve been having really shitty s...,@115714 wtf!? i’ve been having really shitty s...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...
2811771,@143549 @sprintcare You have to go to https://...,@143549 @sprintcare you have to go to https://...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...
2811772,"@823870 Sounds delicious, Sarah! 😋 https://t.c...","@823870 sounds delicious, sarah! 😋 https://t.c...",823870 sounds delicious sarah 😋 httpstco7uqpwy...,823870 sounds delicious sarah httpstco7uqpwyh1b6
2811773,@AldiUK warm sloe gin mince pies with ice cre...,@aldiuk warm sloe gin mince pies with ice cre...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...


<div> Remove URLs </div>

In [9]:
textDF["text_wo_url"] = textDF["text_wo_emojis"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
textDF.tail(5)

C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\3549873081.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  textDF["text_wo_url"] = textDF["text_wo_emojis"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\3549873081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF["text_wo_url"] = textDF["text_wo_emojis"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()


,text,text_lower,text_wo_punctuations,text_wo_emojis,text_wo_url
2811769,"@823869 Hey, we'd be happy to look into this f...","@823869 hey, we'd be happy to look into this f...",823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...
2811770,@115714 wtf!? I’ve been having really shitty s...,@115714 wtf!? i’ve been having really shitty s...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...
2811771,@143549 @sprintcare You have to go to https://...,@143549 @sprintcare you have to go to https://...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...
2811772,"@823870 Sounds delicious, Sarah! 😋 https://t.c...","@823870 sounds delicious, sarah! 😋 https://t.c...",823870 sounds delicious sarah 😋 httpstco7uqpwy...,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6
2811773,@AldiUK warm sloe gin mince pies with ice cre...,@aldiuk warm sloe gin mince pies with ice cre...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...


<div> Remove Stopwords </div>

In [10]:
', '.join(stopwords.words('english'))

stopwords_set = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join(words for words in str(text).split() if words not in stopwords_set)
    

textDF['text_wo_stop'] = textDF['text_wo_url'].apply(lambda text: remove_stopwords(text))
textDF.tail(5)

C:\Users\Crolw\AppData\Local\Temp\ipykernel_6612\1074926294.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textDF['text_wo_stop'] = textDF['text_wo_url'].apply(lambda text: remove_stopwords(text))


,text,text_lower,text_wo_punctuations,text_wo_emojis,text_wo_url,text_wo_stop
2811769,"@823869 Hey, we'd be happy to look into this f...","@823869 hey, we'd be happy to look into this f...",823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed happy look please send us direc...
2811770,@115714 wtf!? I’ve been having really shitty s...,@115714 wtf!? i’ve been having really shitty s...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve really shitty service day get ...
2811771,@143549 @sprintcare You have to go to https://...,@143549 @sprintcare you have to go to https://...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare go httpstcov2tmhetl7q ask ad...
2811772,"@823870 Sounds delicious, Sarah! 😋 https://t.c...","@823870 sounds delicious, sarah! 😋 https://t.c...",823870 sounds delicious sarah 😋 httpstco7uqpwy...,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6
2811773,@AldiUK warm sloe gin mince pies with ice cre...,@aldiuk warm sloe gin mince pies with ice cre...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies ice cream best...


<div> Stem and Lemmatize Text </div>

In [11]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [12]:
# Lemmanize every word
textDF['text_lem_stem'] = textDF['text_wo_stop'].apply(lambda text: ' '.join([lemmatizer.lemmatize(word)
                            if lemmatizer.lemmatize(word).endswith('e') else stemmer.stem(word) for word in text.split()]))

In [13]:
textDF.head(5)

,text,text_lower,text_wo_punctuations,text_wo_emojis,text_wo_url,text_wo_stop,text_lem_stem
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...,115712 i understand i would like to assist you...,115712 i understand i would like to assist you...,115712 i understand i would like to assist you...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...,sprintcare i have sent several private message...,sprintcare i have sent several private message...,sprintcare i have sent several private message...,sprintcare sent several private messages one r...,sprintcare sent sever private message one resp...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...,115712 please send us a private message so tha...,115712 please send us a private message so tha...,115712 please send us a private message so tha...,115712 please send us private message assist c...,115712 please send us private message assist c...
4,@sprintcare I did.,@sprintcare i did.,sprintcare i did,sprintcare i did,sprintcare i did,sprintcare,sprintcare


<div> Remove most common words </div>

In [37]:
cnt = Counter()

for text in textDF['text_lem_stem'].values:
    for word in text.split():
        cnt[word] +=1
        
cnt.most_common(30)

[('us', 451298),
 ('please', 402790),
 ('dm', 341263),
 ('thank', 316000),
 ('help', 297449),
 ('get', 253455),
 ('hi', 224604),
 ('sorri', 192275),
 ('look', 176972),
 ('service', 157183),
 ('issue', 153148),
 ('like', 150205),
 ('let', 149264),
 ('send', 148950),
 ('know', 148298),
 ('well', 137339),
 ('amazonhelp', 137290),
 ('account', 134691),
 ('time', 131686),
 ('im', 131579),
 ('number', 130986),
 ('email', 127271),
 ('work', 125122),
 ('phone', 119681),
 ('need', 119390),
 ('tri', 117073),
 ('order', 114643),
 ('use', 112670),
 ('check', 110033),
 ('go', 109689)]

In [15]:
freqWords = set([w for (w,wc) in cnt.most_common(10)])

def removeFreqwords(text):
    return ' '.join(word for word in str(text).split() if word not in freqWords)

textDF['text_wo_freq'] = textDF['text_lem_stem'].apply(lambda text:removeFreqwords(text))
textDF.tail(5)

,text,text_lower,text_wo_punctuations,text_wo_emojis,text_wo_url,text_wo_stop,text_lem_stem,text_wo_freq
2811769,"@823869 Hey, we'd be happy to look into this f...","@823869 hey, we'd be happy to look into this f...",823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed happy look please send us direc...,823869 hey wed happi look please send us direc...,823869 hey wed happi send direct message assis...
2811770,@115714 wtf!? I’ve been having really shitty s...,@115714 wtf!? i’ve been having really shitty s...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve really shitty service day get ...,115714 wtf i’ve realli shitti service day get ...,115714 wtf i’ve realli shitti day shit togeth
2811771,@143549 @sprintcare You have to go to https://...,@143549 @sprintcare you have to go to https://...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare go httpstcov2tmhetl7q ask ad...,143549 sprintcare go httpstcov2tmhetl7q ask ad...,143549 sprintcare go httpstcov2tmhetl7q ask ad...
2811772,"@823870 Sounds delicious, Sarah! 😋 https://t.c...","@823870 sounds delicious, sarah! 😋 https://t.c...",823870 sounds delicious sarah 😋 httpstco7uqpwy...,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sound delici sarah httpstco7uqpwyh1b6,823870 sound delici sarah httpstco7uqpwyh1b6
2811773,@AldiUK warm sloe gin mince pies with ice cre...,@aldiuk warm sloe gin mince pies with ice cre...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies ice cream best...,aldiuk warm sloe gin mince pie ice cream best ...,aldiuk warm sloe gin mince pie ice cream best ...


<div> Remove Rare Words </div>

In [16]:
numRareWords = 10
rareWords = set([w for (w,c) in cnt.most_common()][:- numRareWords: -1])
print(rareWords)

def removeRarewords(text):
    return ' '.join(word for word in str(text).split() if word not in rareWords)

textDF['text_wo_freqrare'] = textDF['text_wo_freq'].apply(lambda text:removeRarewords(text))
textDF.tail(5)

{'httpstcogfyuq1kjtk', '823869', 'httpstco7uqpwyh1b6', 'httpstco4v1ft0th5x', '823867', '823870', '823868', 'httpstcov2tmhetl7q', 'notjustxmasallyearround'}


,text,text_lower,text_wo_punctuations,text_wo_emojis,text_wo_url,text_wo_stop,text_lem_stem,text_wo_freq,text_wo_freqrare
2811769,"@823869 Hey, we'd be happy to look into this f...","@823869 hey, we'd be happy to look into this f...",823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed be happy to look into this for ...,823869 hey wed happy look please send us direc...,823869 hey wed happi look please send us direc...,823869 hey wed happi send direct message assis...,hey wed happi send direct message assist tn
2811770,@115714 wtf!? I’ve been having really shitty s...,@115714 wtf!? i’ve been having really shitty s...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve been having really shitty serv...,115714 wtf i’ve really shitty service day get ...,115714 wtf i’ve realli shitti service day get ...,115714 wtf i’ve realli shitti day shit togeth,115714 wtf i’ve realli shitti day shit togeth
2811771,@143549 @sprintcare You have to go to https://...,@143549 @sprintcare you have to go to https://...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare you have to go to httpstcov2...,143549 sprintcare go httpstcov2tmhetl7q ask ad...,143549 sprintcare go httpstcov2tmhetl7q ask ad...,143549 sprintcare go httpstcov2tmhetl7q ask ad...,143549 sprintcare go ask add hulu account text...
2811772,"@823870 Sounds delicious, Sarah! 😋 https://t.c...","@823870 sounds delicious, sarah! 😋 https://t.c...",823870 sounds delicious sarah 😋 httpstco7uqpwy...,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sounds delicious sarah httpstco7uqpwyh1b6,823870 sound delici sarah httpstco7uqpwyh1b6,823870 sound delici sarah httpstco7uqpwyh1b6,sound delici sarah
2811773,@AldiUK warm sloe gin mince pies with ice cre...,@aldiuk warm sloe gin mince pies with ice cre...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies with ice crea...,aldiuk warm sloe gin mince pies ice cream best...,aldiuk warm sloe gin mince pie ice cream best ...,aldiuk warm sloe gin mince pie ice cream best ...,aldiuk warm sloe gin mince pie ice cream best ...


<div> Finalize Clean Data </div>

In [17]:
cleanedDF = df.copy()
cleanedDF.insert(5, 'cleaned_text', textDF['text_wo_freqrare'])
cleanedDF.head(5)

,tweet_id,author_id,inbound,created_at,text,cleaned_text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,sprintcare propose,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,sprintcare sent sever private message one resp...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,115712 send private message assist click ‘mess...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,sprintcare,4,6.0


In [18]:
def countWords(text):
    numWords = len(text.split())
    return numWords

cleanedDF.insert(6, 'wordDifference', cleanedDF['text'].apply(countWords) - cleanedDF['cleaned_text'].apply(countWords))
cleanedDF.head(5)


,tweet_id,author_id,inbound,created_at,text,cleaned_text,wordDifference,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,12,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,sprintcare propose,7,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,sprintcare sent sever private message one resp...,6,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,115712 send private message assist click ‘mess...,14,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,sprintcare,2,4,6.0


<div> Download VADER from nltk </div>

In [19]:
#nltk.download()
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Crolw\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

#### Make function to get sentiment data

In [20]:
analyzer = SentimentIntensityAnalyzer()

def getSentiment(text):
    
    scores = analyzer.polarity_scores(str(text))
    
    if scores['compound'] > 0:
        sentiment = 1
    elif scores['compound'] < 0:
        sentiment = -1
    else:
        sentiment = 0

    return sentiment

#### Get sentiment data for each text data
#### Import result into new column

In [21]:
cleanedDF['sentiment'] = cleanedDF['cleaned_text'].apply(getSentiment)

In [22]:
cleanedDF.head(5)

,tweet_id,author_id,inbound,created_at,text,cleaned_text,wordDifference,response_tweet_id,in_response_to_tweet_id,sentiment
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,12,2,3.0,1
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,sprintcare propose,7,NaN,1.0,0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,sprintcare sent sever private message one resp...,6,1,4.0,0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,115712 send private message assist click ‘mess...,14,3,5.0,1
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,sprintcare,2,4,6.0,0


<div> Wrangle Data to show which messages are being sent to the other </div>

In [23]:
firstInbound = cleanedDF[pd.isnull(cleanedDF.in_response_to_tweet_id) & cleanedDF.inbound]

wrangledData = pd.merge(firstInbound, cleanedDF, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')

# Filter to only outbound replies (from companies)
wrangledData = wrangledData[wrangledData.inbound_y ^ True]
print(f'Data shape: {wrangledData.shape}')
wrangledData.head(10)

'''
airDF = pd.DataFrame()
for airline in airlines:
    airDF = pd.concat([airDF, wrangledData[wrangledData['author_id_y'].str.contains(airline)]], ignore_index=True)
'''

Data shape: (794299, 20)


"\nairDF = pd.DataFrame()\nfor airline in airlines:\n    airDF = pd.concat([airDF, wrangledData[wrangledData['author_id_y'].str.contains(airline)]], ignore_index=True)\n"

In [24]:
wrangledData.head(10)

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,cleaned_text_x,wordDifference_x,response_tweet_id_x,in_response_to_tweet_id_x,sentiment_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,cleaned_text_y,wordDifference_y,response_tweet_id_y,in_response_to_tweet_id_y,sentiment_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare worst custom,3,"9,6,10",NaN,-1,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,115712 send private message gain detail account,12,"5,7",8.0,1
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare worst custom,3,"9,6,10",NaN,-1,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,115712 would love chance review account provid...,5,NaN,8.0,1
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,sprintcare worst custom,3,"9,6,10",NaN,-1,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,115712 hello never like custom feel like valu,6,NaN,8.0,-1
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,115714 y’all lie “great” connect 5 bar lte sti...,2,17,NaN,-1,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,115713 h wed definit like work long experienc ...,11,16,18.0,1
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",115714 whenev contact custom support tell shor...,12,19,NaN,1,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,115715 send private message send link access a...,11,NaN,20.0,0
5,29,115716,True,Tue Oct 31 22:01:35 +0000 2017,actually that's a broken link you sent me and ...,actual that broken link sent incorrect inform ...,4,28,NaN,-1,28,Ask_Spectrum,False,Tue Oct 31 22:05:37 +0000 2017,@115716 The information pertaining to the acco...,115716 inform pertain account assumpt correct ...,10,24,29.0,0
6,31,115717,True,Tue Oct 31 22:06:54 +0000 2017,"Yo @Ask_Spectrum, your customer service reps a...",yo askspectrum custom rep super nice— imma sta...,8,30,NaN,1,30,Ask_Spectrum,False,Tue Oct 31 22:12:09 +0000 2017,"@115717 Hello, My apologies for any frustratio...",115717 hello apolog frustrat inconvenience i’d...,11,NaN,31.0,-1
7,33,115718,True,Tue Oct 31 22:06:56 +0000 2017,My picture on @Ask_Spectrum pretty much every ...,picture askspectrum pretti much everi day pay ...,5,32,NaN,-1,32,Ask_Spectrum,False,Tue Oct 31 22:10:08 +0000 2017,@115718 I apologize for the inconvenience. I w...,115718 apologize inconvenience glad assist nam...,17,NaN,33.0,1
8,36,115719,True,Tue Oct 31 22:10:46 +0000 2017,somebody from @VerizonSupport please help meee...,somebodi verizonsupport meeeeee im worst luck ...,9,34,NaN,-1,34,VerizonSupport,False,Tue Oct 31 22:13:33 +0000 2017,@115719 Help has arrived! We are sorry to see ...,115719 arriv see trouble hsb,14,35,36.0,-1
9,39,115720,True,Tue Oct 31 22:12:16 +0000 2017,@VerizonSupport My friend is without internet ...,verizonsupport friend without internet need pl...,6,38,NaN,1,38,VerizonSupport,False,Tue Oct 31 22:13:30 +0000 2017,@115720 Have your friend message us.\n^ACM,115720 friend message acm,3,NaN,39.0,1


<div> Filter Data to only contain Airlines </div>

In [25]:
filteredData = wrangledData[wrangledData['author_id_y'].str.contains('Air|Delta|JetBlue|VirginAtlantic')].drop(wrangledData[wrangledData['author_id_y'] == 'AirbnbHelp'].index)

In [26]:
airlines = filteredData.groupby('author_id_y').nunique().index
print(airlines)

Index(['AirAsiaSupport', 'AlaskaAir', 'AmericanAir', 'British_Airways',
       'Delta', 'JetBlue', 'SouthwestAir', 'VirginAtlantic'],
      dtype='object', name='author_id_y')


In [27]:
filteredData.head(10)

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,cleaned_text_x,wordDifference_x,response_tweet_id_x,in_response_to_tweet_id_x,sentiment_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,cleaned_text_y,wordDifference_y,response_tweet_id_y,in_response_to_tweet_id_y,sentiment_y
76,367,115799,True,Tue Oct 31 10:46:56 +0000 2017,@AirAsiaSupport CAS-19708536-CB92N2 CRM:032660...,airasiasupport cas19708536cb92n2 crm0326600 ch...,9,366,NaN,0,366,AirAsiaSupport,False,Tue Oct 31 13:38:15 +0000 2017,@115799 We have replied to you via DM. Check i...,115799 repli via check yana,8,364,367.0,0
78,371,115801,True,Wed Oct 25 16:02:42 +0000 2017,@115802 @AirAsiaSupport Why are you not refund...,115802 airasiasupport refund trip bali volcano...,7,"372,373,369",NaN,1,372,AirAsiaSupport,False,Wed Oct 25 16:47:02 +0000 2017,"@115801 Hi Ashutosh , Could you provide us the...",115801 ashutosh could provide book number chec...,9,NaN,371.0,1
90,611,115818,True,Sat Aug 06 01:31:50 +0000 2016,@DELTA i booked my flight using delta amex car...,delta book flight use delta amex card check am...,7,609,NaN,0,609,Delta,False,Sat Aug 06 01:44:03 +0000 2016,"@115818 Glad to check. Pls, DM your confirmati...",115818 glad check pl confirm number assistance...,4,610,611.0,1
139,792,115882,True,Tue Oct 31 21:33:48 +0000 2017,@Delta why wasn't earlier flight offered when ...,delta wasnt earlier flight offer tri rebook co...,12,790,NaN,1,790,Delta,False,Tue Oct 31 22:27:38 +0000 2017,@115882 I'm sorry. The earlier flight may not ...,115882 im earlier flight may available time sc...,9,791,792.0,0
140,801,115883,True,Sun Oct 29 15:59:35 +0000 2017,"@Delta The ""change flight"" search option on yo...",delta change flight search option website broken,7,799,NaN,-1,799,Delta,False,Sun Oct 29 16:56:47 +0000 2017,"@115883 I sincerely apologize, our IT team is ...",115883 sincer apologize team dilig work work s...,13,"800,796",801.0,1
141,803,115884,True,Tue Oct 31 21:33:27 +0000 2017,.@delta this has been my inflight studio exper...,delta inflight studio experience today noth wo...,4,802,NaN,0,802,Delta,False,Tue Oct 31 22:23:07 +0000 2017,"@115884 Oh, no! Please speak to a member of th...",115884 oh speak member flt crew immediate assi...,7,NaN,803.0,0
142,814,115885,True,Tue Oct 31 16:37:26 +0000 2017,@Delta I'm flying JFK-MEX-MID tomorrow and you...,delta im fli jfkmexmid tomorrow say im book li...,12,813,NaN,1,813,Delta,False,Tue Oct 31 18:42:21 +0000 2017,"@115885 Hi, Richard. Can you pls DM your confi...",115885 richard pl confirm number may matter tj...,13,811,814.0,1
143,817,115886,True,Tue Oct 31 21:22:28 +0000 2017,@delta #delta moves you to #dm the moment you ...,delta delta move moment complaint result got b...,16,815,NaN,-1,815,Delta,False,Tue Oct 31 22:15:15 +0000 2017,@115886 We sincerely apologize for your disapp...,115886 sincer apologize disappoint alway striv...,6,816,817.0,1
147,872,115892,True,Tue Oct 31 16:08:21 +0000 2017,Hi @British_Airways! My flight from MAN--LHR--...,britishairway flight manlhrbwi nov 3 cancel ex...,24,"867,874",NaN,-1,867,British_Airways,False,Tue Oct 31 22:28:36 +0000 2017,@115892 caused. We're unable to offer a compli...,115892 caus unable offer complimentari upgrade...,10,"868,869,870,871",872.0,1
148,872,115892,True,Tue Oct 31 16:08:21 +0000 2017,Hi @British_Airways! My flight from MAN--LHR--...,britishairway flight manlhrbwi nov 3 cancel ex...,24,"867,874",NaN,-1,874,British_Airways,False,Tue Oct 31 22:26:30 +0000 2017,@115892 We're so sorry to hear your flight was...,115892 hear flight cancel bryce pleas youve re...,14,1980634,872.0,-1


In [34]:
cnt = Counter()

def mostCommon(column):
    for text in column.values:
        for word in text.split():
            cnt[word] +=1

    return cnt.most_common(30)

In [35]:
# Negative Words Dataframe
negativeSentiment = filteredData[filteredData['sentiment_x'] == -1]
negativeWordsDF = pd.DataFrame(mostCommon(negativeSentiment['cleaned_text_x']))
negativeWordsDF.columns = ['CommonWords', 'Count']
negativeWordsDF

,CommonWords,Count
0,flight,16451
1,delta,9233
2,americanair,8934
3,delay,7702
4,britishairway,6518
5,southwestair,5086
6,hour,4024
7,cancel,3456
8,time,3054
9,fli,3004


In [10]:
''' STEP 4: CLEAN THE DATA '''


''' DATA CLEAN HERE '''

# Count number of words in original text and cleaned text

'\nTRANSFORM TEXT DATA INTO SIMPLIFIED FORM\nTRANSFORM DATE\n'

In [6]:
count = df['author_id'].value_counts()
grouped = df.groupby('author_id').nunique()
grouped['count'] = count
sortedGrouped = grouped.sort_values(by=['count'], ascending=False)

In [19]:
inboundSent = df[df['inbound'] == True]
outboundSent = df[df['inbound'] == False]

#### Count the number of values for each sentiment score (Positve, Neutral, Negative)

In [86]:
sentimentCounts = df['sentiment'].value_counts().rename_axis('Sentiment').reset_index(name='count')
sentimentCounts
# test = df[['inbound', 'sentiment']].value_counts().reset_index(name='count')

,Sentiment,count
0,1,1199699
1,0,979997
2,-1,632078


In [117]:
test1 = df.query('inbound == True')[['sentiment', 'inbound']].value_counts().rename_axis(['sentiment', 'inbound']).reset_index(name='count')
test2 = df.query('inbound == False')[['sentiment', 'inbound']].value_counts().rename_axis(['sentiment', 'inbound']).reset_index(name='count')

In [118]:
test1

,sentiment,inbound,count
0,0,True,570332
1,1,True,513961
2,-1,True,453550


In [132]:
merged = test1.merge(test2, left_on='sentiment', right_on='sentiment')
merged['total'] = merged['count_x'] + merged['count_y']
merged = merged.sort_values('sentiment', ascending=False).reset_index()

merged=merged.drop(['index'], axis=1)
merged

,sentiment,inbound_x,count_x,inbound_y,count_y,total
0,1,True,513961,False,685738,1199699
1,0,True,570332,False,409665,979997
2,-1,True,453550,False,178528,632078


In [21]:
inboundSentCounts = inboundSent['sentiment'].value_counts()
outboundSentCounts = outboundSent['sentiment'].value_counts()
print(inboundSentCounts)
print(outboundSentCounts)

 0    570332
 1    513961
-1    453550
Name: sentiment, dtype: int64
 1    685738
 0    409665
-1    178528
Name: sentiment, dtype: int64


#### Plot on graph(s) to display
<p> Green = Positive </p>
<p> Yellow = Neutral </p>
<p> Red = Negative </p>

### Step 5: Exploratory Data Analysis

In [67]:
# Total Sentiments
fig = make_subplots(rows=1, cols=3, subplot_titles=("Inbound + Outbound","Inbound", "Outbound"), horizontal_spacing = 0.25)

fig.add_trace(go.Bar(x=sentimentCounts.index, y=sentimentCounts.values,
                    marker=dict(color=sentimentCounts.index, coloraxis="coloraxis")),
              1, 1)

fig.add_trace(go.Bar(x=inboundSentCounts.index, y=inboundSentCounts.values,
                    marker=dict(color=inboundSentCounts.index, coloraxis="coloraxis")),
              1, 2)

fig.add_trace(go.Bar(x=outboundSentCounts.index, y=outboundSentCounts.values,
                    marker=dict(color=outboundSentCounts.index, coloraxis="coloraxis")),
              1, 3)

fig['layout']['xaxis']['title']='Sentiment'
fig['layout']['xaxis2']['title']='Sentiment'
fig['layout']['xaxis3']['title']='Sentiment'
fig['layout']['yaxis']['title']='Count'
fig['layout']['yaxis2']['title']='Count'
fig['layout']['yaxis3']['title']='Count'

fig.update_layout(height=400, width=1000, yaxis = dict(range=[0, 1500000]), yaxis2 = dict(range=[0, 1500000]), 
                  yaxis3 = dict(range=[0, 1500000]), coloraxis_showscale=False, showlegend=False, 
                  title_text="Sentiment Graphs")
fig.show()

In [75]:
sentimentCounts

 1    1199699
 0     979997
-1     632078
Name: sentiment, dtype: int64

In [73]:
fig = px.bar(sentimentCounts, x=sentimentCounts.index, y=sentimentCounts.values, color=sentimentCounts.values,
             title="Long-Form Input")
fig.show()

#### Wrangle data and sort by Airlines

In [11]:
first_inbound = df[pd.isnull(df.in_response_to_tweet_id) & df.inbound]

QnR = pd.merge(first_inbound, df, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')

# Filter to only outbound replies (from companies)
QnR = QnR[QnR.inbound_y ^ True]
print(f'Data shape: {QnR.shape}')
#QnR.head()

airDF = pd.DataFrame()
for airline in airlines:
    airDF = pd.concat([airDF, QnR[QnR['author_id_y'].str.contains(airline)]], ignore_index=True)

Data shape: (794299, 18)


In [12]:
airDF['in_response_to_tweet_id_y'] = airDF['in_response_to_tweet_id_y'].astype('int64')

In [13]:
airDF.head(10)

,Unnamed: 0_x,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,sentiment_x,Unnamed: 0_y,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y,sentiment_y
0,609,1002,115905,True,2017-10-31,ben tennyson american airline pilot 🎃 trunkort...,1001,NaN,0,608,1001,AmericanAir,False,2017-10-31,115905 aww thats definitely future pilot makin...,NaN,1002,1
1,613,1005,115906,True,2017-10-31,i’m it’s going cost 50 transfer 4000 aa advant...,"1003,1007",NaN,-1,610,1003,AmericanAir,False,2017-10-31,115906 great option customer want share mile c...,1004,1005,1
2,615,1009,115907,True,2017-10-31,heading london tonight 5 day exciting stuff am...,"1008,1010,1011,1012,1013,1014,1015,1016,1017,1...",NaN,1,614,1008,AmericanAir,False,2017-10-31,115907 always pleasure board john wonderful fl...,NaN,1009,1
3,617,1062,115908,True,2017-10-31,check tiny trickortreaters dressed nicu hey am...,"1061,1063",NaN,0,616,1061,AmericanAir,False,2017-10-31,115908 pilot potter pumpkin skeletonsoh theyre...,NaN,1062,1
4,619,1065,115909,True,2017-10-31,thank americanair playing thisisus great fligh...,1064,NaN,1,618,1064,AmericanAir,False,2017-10-31,115909 glad got kick back enjoy show flying ki...,NaN,1065,1
5,1316,1917,116142,True,2017-10-31,americanairs wifi make amtraks wifi look pro l...,1916,NaN,1,1315,1916,AmericanAir,False,2017-10-31,116142 never want experience anything le perfe...,NaN,1917,1
6,1318,1919,116143,True,2017-10-31,wonderful club americanair american airline ad...,1918,NaN,1,1317,1918,AmericanAir,False,2017-10-31,116143 shoutout stopping marc well pas sfo adm...,NaN,1919,1
7,1331,1922,116144,True,2017-10-31,trying book flight americanair error come syst...,"1920,1933",NaN,-1,1319,1920,AmericanAir,False,2017-10-31,116144 difficulty youve experienced contact we...,1921,1922,1
8,1339,1941,116145,True,2017-10-29,look americanair bag httpstcoquebqf8jjf,"1940,1942,1943",NaN,0,1338,1940,AmericanAir,False,2017-10-29,116145 always want take care bag able file cla...,1939,1941,1
9,1341,1945,116146,True,2017-10-31,big thx 2 americanair 4 guacamole margs thx 11...,1944,NaN,1,1340,1944,AmericanAir,False,2017-10-31,116146 happy youre happy sure look delicious,NaN,1945,1


#### Define a function to identify most commonly used words in text

In [20]:
cnt = Counter()

def mostCommon(column):
    for text in column.values:
        for word in text.split():
            cnt[word] +=1

    return cnt.most_common(10)

#### Create Dataframes for different sentiments for airlines

In [24]:
positiveSentiment = airDF[airDF['sentiment_x'] == 1]
negativeSentiment = airDF[airDF['sentiment_x'] == -1]
neutralSentiment = airDF[airDF['sentiment_x'] == 0]

#### Create dataframe to count most common words used in negative sentiments

In [58]:
# Negative Words Dataframe
negativeSentiment = airDF[airDF['sentiment_x'] == -1]
negativeWordsDF = pd.DataFrame(mostCommon(negativeSentiment['text_x']))
negativeWordsDF.columns = ['CommonWords', 'Count']
negativeWordsDF

,CommonWords,Count
0,flight,117441
1,delta,72922
2,americanair,66957
3,britishairways,49978
4,southwestair,41820
5,hour,26958
6,delayed,24954
7,service,24238
8,time,22093
9,seat,20564


#### Display words on graph

In [63]:
fig = px.bar(negativeWordsDF, x='CommonWords', y='Count', title='Common Negative Words', 
             orientation='v', width=500, height=500, color='CommonWords')
fig.show()

#### Create dataframe to count most common words used in positive sentiments

In [60]:
# Positive Words Dataframe
positiveeWordsDF = pd.DataFrame(mostCommon(positiveSentiment['text_x']))
positiveeWordsDF.columns = ['CommonWords', 'Count']
positiveeWordsDF

,CommonWords,Count
0,flight,137886
1,delta,88148
2,americanair,77826
3,britishairways,58664
4,southwestair,53166
5,hour,29430
6,service,28298
7,delayed,26298
8,time,25580
9,seat,23842


#### Display words on graph

In [62]:
fig = px.bar(positiveeWordsDF, x='CommonWords', y='Count', title='Common Positive Words', 
             orientation='v', width=500, height=500, color='CommonWords')
fig.show()

In [ ]:
'''
airDF = pd.DataFrame()
for airline in airlines:
    airDF = pd.concat([airDF, df[df['author_id'].str.contains(airline)]], ignore_index=True)
'''

'''
negativeSentiment = pd.DataFrame()
for i in range(len(testDF['sentiment_x'])):
    negativeSentiment = pd.concat([negativeSentiment, testDF[testDF['sentiment_x'].astype('string').str.contains('-1')]], ignore_index=True)
    
negativeSentiment.head(10)
'''

# SORT DATASET BY SENTIMENT NEGATIVE (-1)
#negativeSentiments = df[['sentiment']]
# SORT THAT DATASET BY INBOUND TWEETS TO SEE WHAT TYPE OF COMPLAINTS CUSTOMERS HAVE
# SORT BY AIRLINES => use author_id

### Step 6: Statistical Prediction/Modeling

In [ ]:
# SOME MACHINE LEARNING
# FIND WHAT TYPE OF METHOD I NEED TO USE TO SOLVE PROBLEM
# EXAMPLES: CLASSIFICATION, REGRESSION

### Step 7: Interpretation of Results

In [ ]:
# EXPLAIN WHAT VARIABLES CORRELATED WITH WHAT
# IF USING MACHINE LEARNING, EXPLAIN WHY SOME OF THE PREDICTIONS

### Step 8: Challenging of Results

In [ ]:
# BEFORE FINALIZING RESULTS
'''
QUESTIONS TO KEEP IN MIND:
Is this technique up to date with the industry, or are there better ways to solve this?
Are there other models or methods I can use to analyze this data?
Have I utilized the data to the best ability? Is there any more data I can collect to provide a more conclusive answer?
and many more…
'''

### Step 9: Communicate Results

In [ ]:
# USE A VISUALIZATION FORMAT TO VISUALIZE RESULTS
# EXAMPLES: TABLEAU, POWERBI, MATPLOT FIGURES

'''
TEMPLATE:
Start with your question and your problem statement
Summarize your analysis into a story.
Include the only essential analysis that adds value to your story and addresses the challenge.
Add “pretty” figures that contribute to the story.
Conclude with a summary of the important findings and any further techniques that can be explored to better answer the question.
'''

### Step 10: Data Product

In [ ]:
# TURN DATA INTO A PRODUCT
# EXAMPLES: DASHBOARDS, STREAMLIT FOR WEB APP